In [8]:
import os
import sys

parent_dir = os.path.abspath("/home/leandre/Projects/BioMoQA_Playground")

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from src.data_pipeline.ipbes.create_ipbes_raw import loading_pipeline_from_raw

In [9]:
datasets = loading_pipeline_from_raw(multi_label=True)

2025-07-08 08:59:32,341 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-08 08:59:32,343 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-08 08:59:32,769 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-08 08:59:32,769 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-08 08:59:32,770 - INFO - 86


Resolving data files:   0%|          | 0/86 [00:00<?, ?it/s]

2025-07-08 08:59:32,909 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-08 08:59:32,909 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-08 08:59:32,910 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-08 08:59:33,057 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-08 08:59:33,058 - INFO - pos_ds features for IAS : {'DOI': Value(dtype='string', id=None), 'Title': Value(dtype='string', id=None), 'Abstract Note': Value(dtype='string', id=None), 'Language': Value(dtype='string', id=None), 'Item Type': Value(dtype='string', id=None)}
2025-07-08 08:59:33,058 - INFO - creating corpus dataset


Resolving data files:   0%|          | 0/2524 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

2025-07-08 08:59:34,485 - INFO - dataset loaded
2025-07-08 08:59:34,486 - INFO - ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr']
2025-07-08 08:59:34,486 - INFO - Concatenating positive datasets...
2025-07-08 08:59:34,610 - INFO - creating raw negative dataset
2025-07-08 08:59:36,470 - INFO - creating raw positive dataset
2025-07-08 08:59:36,485 - INFO - Finished positives and negatives creation pipeline


In [10]:
datasets

([Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type'],
      num_rows: 5251
  }),
  Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type'],
      num_rows: 6281
  }),
  Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type'],
      num_rows: 3109
  })],
 Dataset({
     features: ['title', 'abstract', 'doi'],
     num_rows: 4183082
 }),
 Dataset({
     features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
     num_rows: 4227131
 }))

In [11]:
pos_ds_list=datasets[0]
neg_ds=datasets[1]
corpus_ds=datasets[2]

In [12]:
clean_pos_a=pos_ds_list[0].filter(lambda batch : [batch['doi'][i] is not None and batch['Item Type'][i] == 'journalArticle' for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)

In [13]:
print(clean_pos_a)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type'],
    num_rows: 3829
})


In [23]:
clean_pos_b=clean_pos_a.filter(lambda batch : [batch['abstract'][i] is not None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(clean_pos_b)

Filter (num_proc=20):   0%|          | 0/3829 [00:00<?, ? examples/s]

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type'],
    num_rows: 2253
})


In [ ]:
none_abs=clean_pos_a.filter(lambda batch : [batch['abstract'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(none_abs)

Filter (num_proc=20):   0%|          | 0/3829 [00:00<?, ? examples/s]

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type'],
    num_rows: 1576
})


In [15]:
print(none_abs[0])

{'doi': '10.1007/s10530-006-9026-4', 'title': 'Germination and early growth of <i>Ailanthus</i> and tulip poplar in three levels of forest disturbance', 'abstract': None, 'Item Type': 'journalArticle'}


In [16]:
none_abs_instance=corpus_ds.filter(lambda batch : [batch['display_name'][i] == 'Germination and early growth of Ailanthus< and tulip poplar in three levels of forest disturbance' if batch['display_name'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(none_abs_instance)

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 0
})


In [17]:
pos_doi_set=set(pos_ds_list[0]['doi'])
pos_doi_set.remove(None)
none_abs_instance=corpus_ds.filter(lambda batch : [any(batch['doi'][i].endswith(p_doi) for p_doi in pos_doi_set) if batch['doi'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=30)

In [18]:
print(len(pos_doi_set))

3861


In [19]:
print(none_abs_instance)

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 941
})


In [20]:
print(corpus_ds['doi'][0:10])

['https://doi.org/10.1016/s0016-0032(29)90208-8', 'https://doi.org/10.1016/s0016-0032(29)90638-4', 'https://doi.org/10.1098/rstl.1829.0026', 'https://doi.org/10.1016/s0016-0032(29)90698-0', 'https://doi.org/10.1080/14786442908675166', 'https://doi.org/10.5962/bhl.title.109375', 'https://doi.org/10.1016/s0016-0032(29)90539-1', 'https://doi.org/10.1016/s0016-0032(29)90045-4', 'https://doi.org/10.1016/s0016-0032(29)90465-8', 'https://doi.org/10.1098/rstl.1816.0012']


In [21]:
print(corpus_ds.filter(lambda batch : [batch['id'][i].endswith('W2142524821') if batch['doi'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 0
})


In [22]:
print(corpus_ds['id'][0])

https://openalex.org/W1979751022


In [ ]:
recent_pos=clean_pos_b.filter(lambda batch : [batch["Publication Year"][i]>=2022 if batch["Publication Year"][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(recent_pos)